In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv("zomato.csv")
data_review = dataset['reviews_list']

In [3]:
data_review

0        [('Rated 4.0', 'RATED\n  A beautiful place to ...
1        [('Rated 4.0', 'RATED\n  Had been here for din...
2        [('Rated 3.0', "RATED\n  Ambience is not that ...
3        [('Rated 4.0', "RATED\n  Great food and proper...
4        [('Rated 4.0', 'RATED\n  Very good restaurant ...
                               ...                        
51712    [('Rated 5.0', "RATED\n  Food and service are ...
51713                                                   []
51714                                                   []
51715    [('Rated 4.0', 'RATED\n  Nice and friendly pla...
51716    [('Rated 5.0', 'RATED\n  Great ambience , look...
Name: reviews_list, Length: 51717, dtype: object

In [4]:
x = []
y = []
for row_num in range(51717):
    lst = data_review[row_num].split("('")
    for i in lst:
        if len(i) > 5:
            if i.find("',") != -1:
                single_rev = i.split("',")
                if len(single_rev[0]) > 2:
                    x.append(single_rev[0])
                if len(single_rev[1]) > 2:
                    y.append(single_rev[1])

In [5]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

rating_final = []
review_final = []

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
import re
for loop in range(40000):
    data_x = x[loop]
    data_x = re.sub('[a-zA-Z]'," ",data_x)
    data_x = data_x.split()
    data_x = ''.join(data_x)
    data_x = float(data_x)
    if data_x < 2.5:
        rating_final.append("poor")
    elif data_x >= 2.5 and data_x <= 3.5:
        rating_final.append("average")
    elif data_x > 3.5:
        rating_final.append("good")

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
rating_final = le.fit_transform(rating_final)
rating_final = np.array(rating_final)
rating_final = np.expand_dims(rating_final, axis=1)
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
rates = one.fit_transform(rating_final).toarray()

In [8]:
for loop in range(40000):
    data_y = y[loop]
    data_y = re.sub('[^a-zA-Z]'," ",data_y)
    data_y = data_y.lower()
    data_y = data_y.split()
    data_y = [ps.stem(word) for word in data_y if not word in set(stopwords.words('english'))]
    data_y = ' '.join(data_y)
    review_final.append(data_y)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 20000)
x_final = cv.fit_transform(review_final).toarray()

import pickle
pickle.dump(cv, open('cv.pkl','wb'))

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, rates, test_size = 0.2, random_state = 0)

In [11]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units = 13264, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 2000, kernel_initializer = 'random_uniform', activation = 'relu'))
model.add(Dense(units = 3, kernel_initializer = 'random_uniform', activation = 'softmax'))
model.compile(optimizer = 'adam', loss='categorical_crossentropy', metrics=["mae", "acc"])
model.fit(x_train, y_train, batch_size = 128, epochs = 5)

Epoch 1/5
250/250 [==============================] - 393s 2s/step - loss: 0.4973 - mae: 0.1589 - acc: 0.8322
Epoch 2/5
250/250 [==============================] - 387s 2s/step - loss: 0.1388 - mae: 0.0482 - acc: 0.9537
Epoch 3/5
250/250 [==============================] - 362s 1s/step - loss: 0.0923 - mae: 0.0322 - acc: 0.9693
Epoch 4/5
250/250 [==============================] - 358s 1s/step - loss: 0.0815 - mae: 0.0288 - acc: 0.9722
Epoch 5/5
250/250 [==============================] - 380s 2s/step - loss: 0.0753 - mae: 0.0269 - acc: 0.9739


In [12]:
y_pred = model.predict(x_test)
text = "the food is okay. average place "
text = re.sub('[^a-zA-Z]',' ',text)
text = text.lower()
text = text.split()
text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
text = ' '.join(text)

y_p = model.predict(cv.transform([text]))
model.save("zomato_2_analysis.h5")